In [ ]:
is_mc=False
selected_tracks=True
selected_orientation=''

In [ ]:
import h5py
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
from matplotlib import gridspec
import os

coverage_dir='/global/cfs/cdirs/dune/users/brussell/2x2/detector_properties/dqdx/'
#filedir='/pscratch/sd/c/cuddandr/elife/MiniRun6_ELIFE_10000/run-ndlar-flow/MiniRun6_1E19_RHC.flow/FLOW/0000000/'
filedir='/global/cfs/cdirs/dune/www/data/2x2/nearline/flowed_charge/REFLOW/v4/beam/july10_2024/nominal_hv/'
drift_distance=30.

In [ ]:
def make_x_plane(y1, y2, z1, z2, x):
    y = np.linspace(y1, y2, 100)
    z = np.linspace(z1, z2, 100)
    Y, Z = np.meshgrid(y, z)
    X = np.full(Y.shape, x)
    return X, Y, Z

In [ ]:
def configure_plot():
    fig = plt.figure()
    spec = gridspec.GridSpec(ncols=3, nrows=1, width_ratios=[2,1,1])
    ax = fig.add_subplot(spec[0], projection='3d')
    ax1 = fig.add_subplot(spec[1])
    ax2 = fig.add_subplot(spec[2])
                 
    ax.grid(False)
    ax.xaxis.pane.fill = True
    ax.yaxis.pane.fill = True
    ax.zaxis.pane.fill = True
    for i in range(len(f['geometry_info'].attrs['module_RO_bounds'])):
        X_cathode, Y_cathode, Z_cathode = make_x_plane(f['geometry_info'].attrs['module_RO_bounds'][i][0][1], f['geometry_info'].attrs['module_RO_bounds'][i][1][1], \
                                                           f['geometry_info'].attrs['module_RO_bounds'][i][0][2], f['geometry_info'].attrs['module_RO_bounds'][i][1][2], 
                                                           f['geometry_info'].attrs['module_RO_bounds'][i][0][0]+f['geometry_info'].attrs['max_drift_distance']+f['geometry_info'].attrs['cathode_thickness']/2)
        ax.plot_surface(Z_cathode,X_cathode,Y_cathode, color='gainsboro', alpha=0.1)
        ax1.plot([f['geometry_info'].attrs['module_RO_bounds'][i][0][2], f['geometry_info'].attrs['module_RO_bounds'][i][1][2]],
                     [f['geometry_info'].attrs['module_RO_bounds'][i][0][0]+f['geometry_info'].attrs['max_drift_distance']+\
                      f['geometry_info'].attrs['cathode_thickness']/2, 
                      f['geometry_info'].attrs['module_RO_bounds'][i][0][0]+f['geometry_info'].attrs['max_drift_distance']+\
                      f['geometry_info'].attrs['cathode_thickness']/2],
                     color='gainsboro', alpha=0.75)
        for j in range(2):
            for k in range(2):
                # Plot outlines of modules for XYZ (beam, drift, vertical) 3D view:
                ax.plot([f['geometry_info'].attrs['module_RO_bounds'][i][j][2], f['geometry_info'].attrs['module_RO_bounds'][i][j][2]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][0][0], f['geometry_info'].attrs['module_RO_bounds'][i][1][0]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][k][1], f['geometry_info'].attrs['module_RO_bounds'][i][k][1]], color='black', alpha=0.35)
                ax1.plot([f['geometry_info'].attrs['module_RO_bounds'][i][j][2], f['geometry_info'].attrs['module_RO_bounds'][i][j][2]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][0][0], f['geometry_info'].attrs['module_RO_bounds'][i][1][0]], \
                            color='black', alpha=0.35)
                ax2.plot([f['geometry_info'].attrs['module_RO_bounds'][i][j][2], f['geometry_info'].attrs['module_RO_bounds'][i][j][2]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][0][1], f['geometry_info'].attrs['module_RO_bounds'][i][1][1]], \
                            color='black', alpha=0.35)
                ax.plot([f['geometry_info'].attrs['module_RO_bounds'][i][j][2], f['geometry_info'].attrs['module_RO_bounds'][i][j][2]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][k][0], f['geometry_info'].attrs['module_RO_bounds'][i][k][0]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][0][1], f['geometry_info'].attrs['module_RO_bounds'][i][1][1]], color='black', alpha=0.35)
                ax1.plot([f['geometry_info'].attrs['module_RO_bounds'][i][j][2], f['geometry_info'].attrs['module_RO_bounds'][i][j][2]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][k][0], f['geometry_info'].attrs['module_RO_bounds'][i][k][0]], \
                            color='black', alpha=0.35)
                ax2.plot([f['geometry_info'].attrs['module_RO_bounds'][i][j][2], f['geometry_info'].attrs['module_RO_bounds'][i][j][2]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][k][1], f['geometry_info'].attrs['module_RO_bounds'][i][k][1]], \
                            color='black', alpha=0.35)
                ax.plot([f['geometry_info'].attrs['module_RO_bounds'][i][0][2], f['geometry_info'].attrs['module_RO_bounds'][i][1][2]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][j][0], f['geometry_info'].attrs['module_RO_bounds'][i][j][0]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][k][1], f['geometry_info'].attrs['module_RO_bounds'][i][k][1]], color='black', alpha=0.35)
                ax1.plot([f['geometry_info'].attrs['module_RO_bounds'][i][0][2], f['geometry_info'].attrs['module_RO_bounds'][i][1][2]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][j][0], f['geometry_info'].attrs['module_RO_bounds'][i][j][0]], \
                            color='black', alpha=0.35)
                ax2.plot([f['geometry_info'].attrs['module_RO_bounds'][i][0][2], f['geometry_info'].attrs['module_RO_bounds'][i][1][2]], \
                            [f['geometry_info'].attrs['module_RO_bounds'][i][j][1], f['geometry_info'].attrs['module_RO_bounds'][i][j][1]], \
                            color='black', alpha=0.35)
            
    ax.tick_params(axis='both', which='major') 
    ax.set_xlabel('Z [cm] (Beam Axis)') 
    ax.set_ylabel('X [cm] (Drift Axis)') 
    ax.set_zlabel('Y [cm] (vertical Axis)')
    ax1.set_xlabel('Z [cm] (Beam Axis)') 
    ax1.set_ylabel('X [cm] (Drift Axis)') 
    ax2.set_xlabel('Z [cm] (Beam Axis)') 
    ax2.set_ylabel('Y [cm] (Drift Axis)')
    fig.set_figheight(8)
    fig.set_figwidth(20)
    return fig, ax, ax1, ax2

In [ ]:
for filename in glob.glob(coverage_dir+'*.json'):
    if is_mc==False:
        proxy=filename.split("/")[-1].split("tgm_coverage_")[-1].split(".json")[0]
        flag=False
        if os.path.exists(filedir+'packet-'+proxy+'_CDT.FLOW.hdf5'): flag=True
        if flag==False: continue
        f=h5py.File(filedir+'packet-'+proxy+'_CDT.FLOW.hdf5','r')
    if is_mc==True:
        proxy=filename.split("/")[-1].split("tgm_coverage_mc")[-1].split(".json")[0]
        elifetime=proxy.split("-")[0]
        file_run_no=proxy.split("-")[-1]
        prefix='MiniRun6_1E19_RHC.flow.'
        suffix='.FLOW.hdf5'
        f=h5py.File(filedir+prefix+file_run_no+suffix, 'r')

    with open(filename, 'r') as json_file: d = json.load(json_file)   
    hits_full = f['charge/calib_prompt_hits/data']
    hits_ref = f['charge/events/ref/charge/calib_prompt_hits/ref']
    hits_region = f['charge/events/ref/charge/calib_prompt_hits/ref_region']
    event_ids=np.unique(f['charge/events/data']['id'])
    for event_id in event_ids:
        hit_ref = hits_ref[hits_region[event_id,'start']:hits_region[event_id,'stop']]
        hit_ref = np.sort(hit_ref[hit_ref[:,0] == event_id, 1])
        hits = hits_full[hit_ref]
        if len(hits)<50: continue

        iog_select_x, iog_select_y, iog_select_z=[[] for i in range(3)]
        for orientation in d.keys():
            for tpc_id in d[orientation].keys():
                for event_id_str in d[orientation][tpc_id].keys():
                    if str(event_id)==event_id_str:
                        if d[orientation][tpc_id][event_id_str][0]>0.025: continue
                        if d[orientation][tpc_id][event_id_str][1]>0.025: continue
                        if orientation=='x': iog_select_x.append(int(tpc_id))
                        if orientation=='y': iog_select_y.append(int(tpc_id))
                        if orientation=='z': iog_select_z.append(int(tpc_id))
        if selected_orientation=='' and selected_tracks==True and len(iog_select_x)==0 and len(iog_select_y)==0 and len(iog_select_z)==0: continue
        if selected_orientation=='x' and len(iog_select_x)==0: continue
        if selected_orientation=='y' and len(iog_select_y)==0: continue
        if selected_orientation=='z' and len(iog_select_z)==0: continue
                
        fig, ax, ax1, ax2 = configure_plot()
        for iog in range(1,9,1):
            color='k'
            if iog in iog_select_x: color='b'
            if iog in iog_select_y: color='g'
            if iog in iog_select_z: color='r'
            iog_hits=hits[hits['io_group']==iog]          
            ax.scatter(iog_hits['z'], iog_hits['x'], iog_hits['y'], s=[100*(q/20e3) for q in iog_hits['Q']], c=color)
            ax1.scatter(iog_hits['z'], iog_hits['x'], s=[100*(q/20e3) for q in iog_hits['Q']], c=color)
            ax2.scatter(iog_hits['z'], iog_hits['y'], s=[100*(q/20e3) for q in iog_hits['Q']], c=color)     

        if is_mc==False: ax.set_title('FLOW v4'+'\n'+filename.split("/")[-1].split(".FLOW")[0]+'\n Event ID '+str(event_id))
        if is_mc==True: ax.set_title('MC electric lifetime '+elifetime+' us'+'\n'+'file ID '+proxy+'\n Event ID '+str(event_id))
        plt.show()
        flag = input("If continue: 1, If exit: 0 ")
        if flag==0: break